#Installation des modules pré-requis

In [31]:
%%capture
!pip install praat-parselmouth

#Import des modules nécessaires

In [ ]:
import parselmouth
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import interpolate

#Préprocessing des fichiers audio
Pour utiliser cette partie-là il faut d'abord télécharger des corpus utilisés et de les mettre dans la répértoire "audio" chacune dans sa propre répértoire.

In [32]:
# replace zeroes in numpy array to prevent divide by Zero Warning while using log
def replaceZeroes(data):
  data[data == 0] = np.min(data[np.nonzero(data)])
  return data


## audio to spectro preprocessing
for targetdir in ['fr', "ar", "ru", "hi"]:
  print(targetdir)
  for path in os.listdir(targetdir):
    print(path)
    snd=parselmouth.Sound("audio/"+targetdir+"/"+path)
    spectrogram = snd.to_spectrogram(maximum_frequency=8000)
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    Xbins, Ybins = spectrogram.x_bins(), spectrogram.y_bins()
    Ymid = np.column_stack((Ybins[:,0],np.diff(Ybins,axis=1)/2)).sum(axis=1)
    sg_db = 10 * np.where(spectrogram.values > 0.0000000001, np.log10(replaceZeroes(spectrogram.values)), -10)
    fig = plt.figure(frameon=False)
    fig.set_size_inches(0.25,0.25)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)


    sg_bd_interp = interpolate.RectBivariateSpline(np.column_stack((Xbins[:,0],np.diff(Xbins,axis=1)/2)).sum(axis=1), Ymid, sg_db.T)(np.linspace(X[0], X[-1], 10*len(X)), np.linspace(Y[0], Y[-1], 10*len(Y))).T

    # add min value in image on the right of the image if zero_padding_width_ratio is over 0 and below 1
    zero_padding_width_ratio=(2-snd.get_total_duration())/2
    if 0 < zero_padding_width_ratio < 1:
      value_for_zero_padding=np.min(sg_bd_interp)
      zero_padded_sg_bd_interp = np.concatenate((sg_bd_interp,np.ones((sg_bd_interp.shape[0], int(np.round(sg_bd_interp.shape[1] * zero_padding_width_ratio / (1 - zero_padding_width_ratio)))))*value_for_zero_padding), axis = 1)
    else:
      zero_padded_sg_bd_interp = sg_bd_interp

    ax.imshow(zero_padded_sg_bd_interp, aspect="auto", cmap="gray_r", vmin=sg_db.max() - 70, vmax=None)
    fig.savefig("imagesSpectros/"+targetdir+"/"+path[:-3]+"png", dpi = 200)
    plt.close(fig)  

fr
common_voice_fr_17377399.mp3
